# Day 1, Session 2: ReAct Pattern in Action

## Building Intelligent Agents with Reasoning + Acting

### What Makes an Agent Different?

Traditional programming follows this pattern:
```python
def process_invoice(invoice_id):
    data = extract_data(invoice_id)
    validation = validate_rules(data)
    return make_decision(validation)
```

But what if you don't know ahead of time what steps you'll need? What if the process depends on what you discover along the way?

**Agents solve this by reasoning about what to do next:**

```python
# Agent pattern
while not done:
    thought = llm.think("What should I do next?")
    action = choose_action(thought)
    observation = execute_action(action)
    context += observation
```

### The ReAct Pattern: Reasoning + Acting

ReAct combines the reasoning capabilities of Large Language Models with the precision of programmatic tools:

```
Input: "Process invoice INV-2024-001"

Step 1:
THOUGHT: I need to first get the invoice data to understand what I'm working with
ACTION: extract_invoice_data("INV-2024-001")
OBSERVATION: {"vendor": "TechCorp", "amount": 15000, "terms": "Net 30"}

Step 2:
THOUGHT: Now I need to check if these payment terms are acceptable for this amount
ACTION: validate_payment_terms("Net 30", 15000)
OBSERVATION: {"approved": true, "reason": "Standard terms for this amount"}

Step 3:
THOUGHT: Everything looks good, I can approve this invoice
ACTION: FINISH
RESULT: "Invoice approved for payment"
```

### Why This Matters for Invoice Processing

**Traditional automation breaks when:**
- Invoice formats vary unexpectedly
- New business rules need to be applied
- Exception handling requires human-like reasoning

**Agents handle this by:**
- Adapting their approach based on what they discover
- Combining multiple tools intelligently
- Explaining their reasoning for auditability

Let's build this step by step!

In [ ]:
# Download real invoice and receipt images first
import requests
import zipfile
import io
import os

# Dropbox shared link for the folder
dropbox_url = "https://www.dropbox.com/scl/fo/m9hyfmvi78snwv0nh34mo/AMEXxwXMLAOeve-_yj12ck8?rlkey=urinkikgiuven0fro7r4x5rcu&st=hv3of7g7&dl=1"

print(f"Downloading real invoice data from: {dropbox_url}")

try:
    response = requests.get(dropbox_url)
    response.raise_for_status()

    # Read the content as a zip file
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Extract all contents to a directory named 'downloaded_images'
        z.extractall("downloaded_images")

    print("✅ Downloaded and extracted images to 'downloaded_images' folder.")
    
    # List downloaded files
    for root, dirs, files in os.walk("downloaded_images"):
        for file in files:
            print(f"  📄 {os.path.join(root, file)}")

except Exception as e:
    print(f"❌ Error downloading images: {e}")

# Global configuration - Instructor will fill these
OLLAMA_URL = "http://XX.XX.XX.XX"  # Course server IP (port 80 via nginx)
API_TOKEN = "YOUR_TOKEN_HERE"      # Instructor provides token
MODEL = "qwen3:8b"                  # Default model on server

# For testing locally, you can use these values:
# OLLAMA_URL = "http://localhost:11434"  # If running Ollama locally
# API_TOKEN = "test-token"
# MODEL = "llama2"  # Or any model you have locally

## Step 1: Setting Up LLM Communication

### The Foundation: Talking to Language Models

Before we can build agents, we need reliable communication with our LLM. Here's the pattern for enterprise LLM integration:

```python
# Enterprise LLM Integration Pattern
def call_llm(prompt, model="qwen3:8b"):
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # For production, consider streaming
    }
    
    response = requests.post(f"{OLLAMA_URL}/think", 
                           headers=headers, json=payload)
    return response.json()["response"]
```

### Health Checks and Error Handling

Production systems need robust health monitoring:

```python
# Health check pattern for LLM services
def check_server_health():
    try:
        response = requests.get(f"{OLLAMA_URL}/health", timeout=5)
        if response.status_code == 200:
            data = response.json()
            return {
                "healthy": True,
                "models_loaded": data.get("models_count", 0),
                "status": data.get("status", "unknown")
            }
    except requests.exceptions.Timeout:
        return {"healthy": False, "error": "timeout"}
    except Exception as e:
        return {"healthy": False, "error": str(e)}
```

Let's test our connection to the course server:

In [ ]:
import requests
import json
from datetime import datetime, timedelta
import time

# Check server health
def check_server_health():
    """Check if the LLM server is healthy and has models loaded"""
    try:
        response = requests.get(f"{OLLAMA_URL}/health")
        if response.status_code == 200:
            data = response.json()
            print("✅ Server Status:", data.get('status', 'Unknown'))
            print("📊 Models Available:", data.get('models_count', 0))
            return True
    except Exception as e:
        print(f"❌ Error connecting to server: {e}")
    return False

# Create LLM calling function
def call_llm(prompt, model=MODEL):
    """Call the LLM with a prompt and return the response"""
    headers = {
        "Authorization": f"Bearer {API_TOKEN}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": model,
        "prompt": prompt
    }
    
    try:
        response = requests.post(
            f"{OLLAMA_URL}/think",
            headers=headers,
            json=data
        )
        if response.status_code == 200:
            return response.json().get('response', '')
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error calling LLM: {e}"

# Test connection
print("🔌 Checking server connection...")
server_available = check_server_health()

if server_available:
    print("\n🧠 Testing LLM with simple prompt...")
    test_response = call_llm("Hello! Please respond with exactly: 'I am ready to process invoices.'")
    print("LLM Response:", test_response)
    
    # Try a more complex question to show reasoning capability
    print("\n🤔 Testing reasoning capability...")
    reasoning_prompt = """Analyze this scenario: An invoice from TechSupplies Co. for $15,000 with Net 30 payment terms. 
    Should this be auto-approved? Explain your reasoning in 2-3 sentences."""
    
    reasoning_response = call_llm(reasoning_prompt)
    print("Reasoning Response:")
    print(reasoning_response[:300] + "..." if len(reasoning_response) > 300 else reasoning_response)
    
else:
    print("\n⚠️ Server not available. Using mock responses for demo.")
    print("💡 In a real deployment, you would:")
    print("   1. Configure the OLLAMA_URL with your server's IP")
    print("   2. Set the API_TOKEN provided by the instructor") 
    print("   3. Verify the MODEL name matches what's loaded on the server")

## Step 2: Building Agent Tools - The Foundation of Intelligence

### The Tool Pattern in AI Agents

Agents become intelligent by combining reasoning with precise tool execution. Here's how to design effective agent tools:

```python
# Tool Design Pattern
def agent_tool(input_data):
    """
    Clear docstring - LLM uses this to understand the tool
    
    Args:
        input_data: Specific type description
    
    Returns:
        Structured response with status and data
    """
    try:
        # 1. Validate input
        # 2. Execute core logic
        # 3. Return structured result
        return {"status": "success", "data": result}
    except Exception as e:
        return {"status": "error", "message": str(e)}
```

### Business Logic Encapsulation

Invoice processing requires specific business logic. Here's how to structure it:

```python
# Business Rules Pattern
BUSINESS_RULES = {
    "payment_terms": {
        "Net 30": {"max_amount": None, "approval_level": "auto"},
        "Net 60": {"max_amount": 10000, "approval_level": "manager"},
        "Net 90": {"max_amount": None, "approval_level": "cfo"}
    },
    "amount_thresholds": {
        "auto_approve": 5000,
        "manager_approval": 25000,
        "cfo_approval": 100000
    }
}

def validate_against_rules(invoice_data, rules=BUSINESS_RULES):
    # Apply business logic systematically
    pass
```

### Data Modeling for Invoice Processing

Structured data is crucial for reliable agent decisions:

```python
# Invoice Data Model
{
    "invoice_id": "INV-2024-001",
    "vendor": {
        "name": "TechSupplies Co.",
        "vat_number": "GB123456789",
        "status": "approved"
    },
    "financial": {
        "amount": 15000.00,
        "currency": "USD",
        "payment_terms": "Net 30"
    },
    "dates": {
        "invoice_date": "2024-01-15",
        "due_date": "2024-02-14"
    },
    "line_items": [
        {"description": "Laptops", "quantity": 5, "unit_price": 2000}
    ]
}
```

Let's implement these patterns:

In [ ]:
# Mock Invoice Processing Tools

def extract_invoice_data(invoice_id):
    """Extract data from an invoice (mock implementation)"""
    # Simulate extracting invoice data
    mock_invoices = {
        "INV-2024-001": {
            "invoice_id": "INV-2024-001",
            "vendor": "TechSupplies Co.",
            "amount": 15000.00,
            "currency": "USD",
            "date": "2024-01-15",
            "payment_terms": "Net 30",
            "vat_number": "GB123456789",
            "line_items": [
                {"description": "Laptops", "quantity": 5, "unit_price": 2000},
                {"description": "Software Licenses", "quantity": 10, "unit_price": 500}
            ]
        },
        "INV-2024-002": {
            "invoice_id": "INV-2024-002",
            "vendor": "CloudServices Inc.",
            "amount": 8500.00,
            "currency": "USD",
            "date": "2024-02-01",
            "payment_terms": "Net 60",
            "vat_number": "US987654321"
        }
    }
    
    if invoice_id in mock_invoices:
        return {"status": "success", "data": mock_invoices[invoice_id]}
    else:
        return {"status": "error", "message": f"Invoice {invoice_id} not found"}

def check_payment_terms(terms, amount):
    """Verify payment terms against business rules"""
    # Business rules:
    # - Net 30 for amounts < 10000
    # - Net 60 allowed for amounts > 10000
    # - Net 90 requires special approval
    
    if terms == "Net 90":
        return {"approved": False, "reason": "Net 90 requires CFO approval"}
    elif terms == "Net 60" and amount <= 10000:
        return {"approved": False, "reason": "Net 60 only allowed for amounts > $10,000"}
    elif terms == "Net 30":
        return {"approved": True, "reason": "Standard payment terms"}
    elif terms == "Net 60" and amount > 10000:
        return {"approved": True, "reason": "Extended terms approved for large amount"}
    else:
        return {"approved": False, "reason": f"Unknown payment terms: {terms}"}

def calculate_due_date(invoice_date, payment_terms):
    """Calculate payment due date based on terms"""
    try:
        date = datetime.strptime(invoice_date, "%Y-%m-%d")
        
        # Extract days from payment terms
        if "Net" in payment_terms:
            days = int(payment_terms.split()[1])
            due_date = date + timedelta(days=days)
            return {"due_date": due_date.strftime("%Y-%m-%d"), "days_until_due": days}
        else:
            return {"error": "Invalid payment terms format"}
    except Exception as e:
        return {"error": str(e)}

def search_vendor_history(vendor_name):
    """Search for vendor payment history (mock database)"""
    mock_history = {
        "TechSupplies Co.": {
            "status": "trusted",
            "total_invoices": 45,
            "on_time_payments": 43,
            "average_amount": 12000,
            "last_invoice_date": "2023-12-15"
        },
        "CloudServices Inc.": {
            "status": "new",
            "total_invoices": 2,
            "on_time_payments": 2,
            "average_amount": 7500,
            "last_invoice_date": "2024-01-01"
        },
        "Unknown Vendor": {
            "status": "not_found",
            "message": "No history found for this vendor"
        }
    }
    
    return mock_history.get(vendor_name, mock_history["Unknown Vendor"])

def validate_vat_number(vat_number):
    """Validate VAT number format and checksum (simplified)"""
    # Simple validation - in reality would use EU VIES service
    if vat_number.startswith("GB") and len(vat_number) == 11:
        return {"valid": True, "country": "United Kingdom", "company": "Verified Company"}
    elif vat_number.startswith("US") and len(vat_number) == 11:
        return {"valid": True, "country": "United States", "company": "Verified US Company"}
    else:
        return {"valid": False, "reason": "Invalid VAT number format"}

# Test tools
print("Testing invoice extraction...")
test_invoice = extract_invoice_data("INV-2024-001")
print(f"Invoice data: {json.dumps(test_invoice['data'], indent=2)[:300]}...")

## Step 3: The ReAct Agent Architecture

### Core Agent Components

A production ReAct agent has four key components:

```python
class ReActAgent:
    def __init__(self, llm_func, tools, max_steps=10):
        self.llm_func = llm_func      # Language model interface
        self.tools = tools            # Available actions
        self.max_steps = max_steps    # Safety limit
        self.history = []             # Execution trace
```

### Prompt Engineering for ReAct

The prompt structure is critical for reliable agent behavior:

```python
# ReAct Prompt Template
REACT_PROMPT = """You are an intelligent agent processing invoices.

Task: {task}

Available tools:
{tool_descriptions}

Previous observations:
{observations}

Think step by step:
1. What information do I have?
2. What do I need to find out?
3. Which tool will help me?

Respond in this EXACT format:
THOUGHT: [your reasoning]
ACTION: [tool_name or FINISH]
PARAMS: [parameters or final answer]
"""
```

### Response Parsing Patterns

Reliable parsing is essential for agent execution:

```python
def parse_llm_response(response):
    """
    Parse structured LLM response into actionable components
    """
    parsed = {"thought": "", "action": "", "params": ""}
    
    # Pattern matching for structured responses
    import re
    
    thought_match = re.search(r'THOUGHT:\s*(.*?)(?=ACTION:|$)', response, re.DOTALL)
    action_match = re.search(r'ACTION:\s*(.*?)(?=PARAMS:|$)', response, re.DOTALL)
    params_match = re.search(r'PARAMS:\s*(.*?)$', response, re.DOTALL)
    
    if thought_match:
        parsed["thought"] = thought_match.group(1).strip()
    if action_match:
        parsed["action"] = action_match.group(1).strip()
    if params_match:
        parsed["params"] = params_match.group(1).strip()
    
    return parsed
```

### Execution Loop with Error Handling

The main agent loop handles both success and failure cases:

```python
def run_agent(task):
    observations = ""
    
    for step in range(max_steps):
        # 1. Generate reasoning
        response = llm_func(build_prompt(task, observations))
        parsed = parse_response(response)
        
        # 2. Execute action
        if parsed["action"] == "FINISH":
            return parsed["params"]
        
        try:
            result = execute_tool(parsed["action"], parsed["params"])
            observations += f"Step {step+1}: {result}\n"
        except Exception as e:
            observations += f"Step {step+1}: ERROR - {str(e)}\n"
    
    return "Max steps reached"
```

Let's implement this architecture:

In [ ]:
class ReActAgent:
    def __init__(self, llm_func, tools, max_steps=10):
        self.llm_func = llm_func
        self.tools = tools
        self.max_steps = max_steps
        self.history = []
        
    def think(self, task, observations=""):
        """Use LLM to reason about next action"""
        
        # Build tool descriptions
        tool_descriptions = "\n".join([
            f"- {name}: {func.__doc__}" 
            for name, func in self.tools.items()
        ])
        
        prompt = f"""You are an invoice processing agent using ReAct pattern.

Task: {task}

Previous observations:
{observations}

Available tools:
{tool_descriptions}
- FINISH: Complete with final answer

Think step by step about what to do next. Consider:
1. What information do I already have?
2. What information do I still need?
3. Which tool can help me get that information?

Respond EXACTLY in this format:
THOUGHT: [your reasoning about what to do next]
ACTION: [tool_name or FINISH]
PARAMS: [parameters as JSON or final answer if FINISH]
"""
        
        response = self.llm_func(prompt)
        return self.parse_response(response)
    
    def parse_response(self, response):
        """Parse structured response from LLM"""
        thought = ""
        action = ""
        params = ""
        
        lines = response.strip().split('\n')
        for line in lines:
            if line.startswith("THOUGHT:"):
                thought = line.replace("THOUGHT:", "").strip()
            elif line.startswith("ACTION:"):
                action = line.replace("ACTION:", "").strip()
            elif line.startswith("PARAMS:"):
                params = line.replace("PARAMS:", "").strip()
        
        return {"thought": thought, "action": action, "params": params}
    
    def act(self, action, params):
        """Execute the chosen action"""
        if action == "FINISH":
            return {"type": "final_answer", "result": params}
        
        if action in self.tools:
            try:
                # Parse parameters if they're JSON
                if params.startswith("{"):
                    params_dict = json.loads(params)
                    result = self.tools[action](**params_dict)
                else:
                    # Single parameter
                    result = self.tools[action](params.strip('"'))
                return {"type": "observation", "result": result}
            except Exception as e:
                return {"type": "error", "result": f"Error executing {action}: {str(e)}"}
        else:
            return {"type": "error", "result": f"Unknown action: {action}"}
    
    def run(self, task):
        """Run the agent on a task"""
        observations = ""
        
        print(f"\n📋 TASK: {task}\n")
        print("=" * 80)
        
        for step in range(self.max_steps):
            print(f"\n🔄 Step {step + 1}:")
            
            # Think
            response = self.think(task, observations)
            print(f"\n💭 THOUGHT: {response['thought']}")
            print(f"⚡ ACTION: {response['action']}")
            print(f"📝 PARAMS: {response['params']}")
            
            # Act
            result = self.act(response['action'], response['params'])
            
            if result['type'] == 'final_answer':
                print(f"\n✅ FINAL ANSWER: {result['result']}")
                return result['result']
            else:
                print(f"\n👁️ OBSERVATION: {json.dumps(result['result'], indent=2)[:500]}")
                observations += f"\nStep {step + 1} - {response['action']}: {json.dumps(result['result'])}\n"
        
        print("\n⚠️ Max steps reached without completion")
        return None

# Create agent with our tools
tools = {
    "extract_invoice": extract_invoice_data,
    "check_terms": lambda terms, amount: check_payment_terms(terms, amount),
    "calculate_due": lambda date, terms: calculate_due_date(date, terms),
    "search_vendor": search_vendor_history,
    "validate_vat": validate_vat_number
}

# For demo, use mock LLM if server not available
def mock_llm(prompt):
    """Mock LLM responses for demo purposes"""
    if "Previous observations:" in prompt and "Step 1" not in prompt:
        if "Step 2" in prompt:
            return """THOUGHT: I have the invoice data. Now I need to verify if the payment terms are acceptable for this amount.
ACTION: check_terms
PARAMS: {"terms": "Net 30", "amount": 15000}"""
        elif "Step 3" in prompt:
            return """THOUGHT: The payment terms are approved. Now I should calculate the due date.
ACTION: calculate_due
PARAMS: {"date": "2024-01-15", "terms": "Net 30"}"""
        elif "Step 4" in prompt:
            return """THOUGHT: I have the due date. Let me check the vendor history to assess risk.
ACTION: search_vendor
PARAMS: "TechSupplies Co."""
        else:
            return """THOUGHT: I have all the information needed. The invoice is valid with approved payment terms.
ACTION: FINISH
PARAMS: Invoice INV-2024-001 is approved. Amount: $15,000, Payment terms: Net 30 (approved), Due date: 2024-02-14, Vendor: TechSupplies Co. (trusted vendor with good payment history)"""
    else:
        return """THOUGHT: I need to first extract the invoice data to understand what we're dealing with.
ACTION: extract_invoice
PARAMS: "INV-2024-001"""

# Use real LLM if available, otherwise mock
llm_function = call_llm if check_server_health() else mock_llm

print("\n✅ Agent created and ready!")

## Step 4: Agent in Action - Live Reasoning

### Observing the Thought Process

When you run an agent, you see the actual reasoning unfold. This is powerful for:

**Debugging and Optimization:**
```
THOUGHT: The invoice amount is $15,000 which exceeds our auto-approval limit
ACTION: check_approval_requirements  
PARAMS: {"amount": 15000, "vendor": "TechSupplies Co."}
```

**Auditability and Compliance:**
```
THOUGHT: Vendor has 95% on-time payment rate, low risk for extended terms
ACTION: approve_payment_terms
PARAMS: {"terms": "Net 30", "risk_level": "low"}
```

**Error Recovery:**
```
THOUGHT: The VAT number validation failed, but I can proceed with manual review flag
ACTION: flag_for_manual_review
PARAMS: {"reason": "VAT validation failed", "priority": "medium"}
```

### Agent vs Traditional Programming

**Traditional Approach:**
```python
def process_invoice_traditional(invoice_id):
    # Fixed sequence of steps
    data = extract_invoice(invoice_id)
    if not data:
        return "Error: Invoice not found"
    
    validation = validate_rules(data)
    if not validation.passed:
        return "Error: Validation failed"
    
    return approve_invoice(data)
```

**Agent Approach:**
```python
# Adaptive sequence based on discoveries
Step 1: Extract invoice → Found complex line items
Step 2: Validate line items individually → Some failed
Step 3: Flag problematic items → Continue with valid ones
Step 4: Partial approval with notes → Success with caveats
```

The agent adapts its strategy based on what it discovers!

In [ ]:
# Create and run the agent
agent = ReActAgent(llm_function, tools)

# Process an invoice
task = "Process invoice INV-2024-001 and verify it meets our payment policies. Check all relevant information including vendor history and VAT validation."

result = agent.run(task)

## Understanding the ReAct Pattern Through Mock Execution

### How the ReAct Pattern Manifests

Notice how the mock LLM demonstrates the core ReAct pattern cycle:

**Step 1 - Initial Reasoning:**
```
THOUGHT: I need to first extract the invoice data to understand what we're dealing with.
ACTION: extract_invoice
PARAMS: "INV-2024-001"
```

**Step 2 - Adapting to Observations:**
```
THOUGHT: I have the invoice data. Now I need to verify if the payment terms are acceptable.
ACTION: check_terms
PARAMS: {"terms": "Net 30", "amount": 15000}
```

**Step 3 - Building on Previous Steps:**
```
THOUGHT: The payment terms are approved. Now I should calculate the due date.
ACTION: calculate_due
PARAMS: {"date": "2024-01-15", "terms": "Net 30"}
```

### Key ReAct Pattern Elements

**1. Context Awareness:**
Each step builds on previous observations. Notice how the mock_llm checks for "Step 1", "Step 2" in the prompt to determine what has already been discovered.

**2. Structured Decision Making:**
Every action is preceded by explicit reasoning about what information is needed next.

**3. Tool Integration:**
The agent doesn't just reason - it takes concrete actions using available tools to gather real data.

**4. Adaptive Flow:**
Unlike rigid programming, the agent's next step depends on what it observes, allowing for flexible problem-solving.

### Why This Works Better Than Pure LLM

**Pure LLM Response:**
"I would process the invoice by checking the data, validating terms, and calculating due dates."
*(Describes what it would do, but can't actually do it)*

**ReAct Agent Response:**
Actually extracts real data → Validates actual terms → Calculates precise due dates → Provides actionable result
*(Does the work and provides verifiable results)*

This is the power of combining reasoning with execution!

## Step 5: The Power of Reasoning vs Pure LLM

### Understanding the Limitations

**Pure LLM without tools:**
```
User: "Process invoice INV-2024-001"
LLM: "To process this invoice, I would need to:
1. Extract the data from the invoice
2. Validate the payment terms
3. Check the vendor history
4. Calculate the due date
However, I cannot actually access your systems to do this."
```

**Agent with tools:**
```
User: "Process invoice INV-2024-001"
Agent: 
THOUGHT: I need to get the actual invoice data first
ACTION: extract_invoice
OBSERVATION: {"vendor": "TechCorp", "amount": 15000...}

THOUGHT: Now I'll validate the payment terms for this amount
ACTION: validate_terms  
OBSERVATION: {"approved": true, "reason": "Standard terms"}

RESULT: "Invoice INV-2024-001 approved for $15,000 payment"
```

### The Value of Executable Intelligence

**Hallucination vs Reality:**
- LLM might say: "This vendor has good payment history" (possibly hallucinated)
- Agent checks: `search_vendor("TechCorp")` → Returns actual payment data

**Adaptive Processing:**
- LLM gives one-size-fits-all advice
- Agent adapts based on actual data discovered during execution

**Auditability:**
- LLM reasoning is internal and opaque
- Agent creates step-by-step audit trail of decisions and data used

### When to Use Each Approach

**Use Pure LLM for:**
- Content generation and summarization
- Initial analysis and recommendations
- Creative tasks and brainstorming

**Use Agents for:**
- Multi-step workflows with dependencies
- Tasks requiring real data access
- Processes needing auditability
- Complex business logic application

In [ ]:
print("=" * 80)
print("COMPARISON: Agent vs Simple LLM")
print("=" * 80)

# Simple LLM approach
print("\n📱 SIMPLE LLM APPROACH:")
print("-" * 40)

simple_prompt = f"""How would you process invoice INV-2024-001 and verify it meets payment policies?"""

simple_response = """To process invoice INV-2024-001, I would:
1. Extract the invoice data to see the amount and terms
2. Check if the payment terms align with company policies
3. Calculate the due date based on the terms
4. Verify the vendor's payment history
5. Validate the VAT number

However, I cannot actually access the invoice data or execute these checks.
I can only describe the process."""

print("Question:", simple_prompt)
print("\nResponse:", simple_response)

# Agent approach
print("\n\n🤖 AGENT APPROACH:")
print("-" * 40)
print("The agent actually:")
print("✓ Extracted real invoice data: $15,000 from TechSupplies Co.")
print("✓ Verified payment terms: Net 30 approved for this amount")
print("✓ Calculated due date: 2024-02-14")
print("✓ Checked vendor history: Trusted vendor with 95% on-time payment rate")
print("✓ Provided actionable result: Invoice approved for processing")

print("\n💡 KEY DIFFERENCE:")
print("- Simple LLM: Can only describe what it would do")
print("- Agent: Actually executes tools and verifies real data")
print("- Result: Agent provides accurate, actionable information")

## Step 6: Error Handling and Recovery Patterns

### The Reality of Production Systems

In production, things go wrong. Agents need to handle:

**Data Issues:**
```python
# Agent encounters missing data
THOUGHT: The invoice is missing a vendor VAT number
ACTION: check_vendor_database
OBSERVATION: {"vendor_found": false}

THOUGHT: I'll flag this for manual review and continue with other checks
ACTION: flag_for_review
PARAMS: {"issue": "missing_vat", "severity": "medium"}
```

**Service Failures:**
```python
# External service is down
THOUGHT: I need to validate the VAT number
ACTION: validate_vat
OBSERVATION: {"error": "VAT service unavailable"}

THOUGHT: I'll use a fallback validation and note the limitation
ACTION: apply_fallback_validation
PARAMS: {"method": "format_check_only"}
```

### Error Recovery Strategies

**Graceful Degradation:**
```python
# Full validation path
primary_validation → secondary_validation → manual_review

# Degraded path when services fail
basic_validation → flag_for_later_review → conditional_approval
```

**Context-Aware Error Messages:**
```python
# Bad error handling
return "Error: Validation failed"

# Good error handling  
return {
    "status": "partial_success",
    "approved_amount": 12000,
    "flagged_amount": 3000,
    "reason": "Line item #3 requires manual review due to VAT service outage",
    "next_action": "Send to accounts payable for VAT verification"
}
```

### Building Resilient Agents

**Timeout Handling:**
```python
def call_external_service(data, timeout=30):
    try:
        return service.call(data, timeout=timeout)
    except TimeoutError:
        return {"status": "timeout", "fallback_available": True}
```

**Retry Logic:**
```python
THOUGHT: The vendor lookup failed, let me try with a simplified search
ACTION: search_vendor_fuzzy
PARAMS: {"name": "TechCorp", "threshold": 0.8}
```

In [ ]:
print("=" * 80)
print("ERROR HANDLING DEMONSTRATION")
print("=" * 80)

# Try to process a non-existent invoice
error_task = "Process invoice INV-2024-999 and verify payment terms."

# Create a modified mock LLM for error handling
def error_handling_llm(prompt):
    if "Previous observations:" in prompt:
        if "error" in prompt.lower() and "not found" in prompt.lower():
            return """THOUGHT: The invoice INV-2024-999 was not found. I should report this issue and suggest checking the invoice ID.
ACTION: FINISH
PARAMS: Error: Invoice INV-2024-999 not found in the system. Please verify the invoice ID and try again."""
    return """THOUGHT: I need to extract the invoice data first.
ACTION: extract_invoice
PARAMS: "INV-2024-999"""

# Run agent with error scenario
error_agent = ReActAgent(error_handling_llm, tools, max_steps=3)
error_result = error_agent.run(error_task)

print("\n" + "=" * 80)
print("✅ The agent successfully handled the error by:")
print("1. Attempting to extract the invoice")
print("2. Recognizing the error in the observation")
print("3. Providing a helpful error message")
print("4. Suggesting corrective action")

## Step 7: Performance and Production Considerations

### Understanding Agent Costs

**Token Economics:**
```python
# Typical invoice processing agent execution
Step 1: ~500 tokens (initial reasoning + invoice extraction)
Step 2: ~300 tokens (validation reasoning)  
Step 3: ~250 tokens (vendor check reasoning)
Step 4: ~200 tokens (final decision)

Total: ~1,250 tokens per invoice
Cost: ~$0.00125 at $0.001/1K tokens
```

**Optimization Strategies:**
```python
# Bad: Verbose reasoning
THOUGHT: I need to extract the invoice data because without the invoice data I cannot proceed with the validation process and I need to understand what vendor this is and what amount we're dealing with...

# Good: Concise reasoning  
THOUGHT: Need invoice data to start validation
ACTION: extract_invoice
```

### Caching and Performance Patterns

**Vendor History Caching:**
```python
# Cache frequently accessed vendor data
@lru_cache(maxsize=1000)
def get_vendor_history(vendor_name):
    return database.query_vendor_history(vendor_name)
```

**Batch Processing:**
```python
# Process multiple invoices with shared context
def process_invoice_batch(invoice_ids):
    # Load vendor data once for the batch
    vendors = preload_vendor_data(invoice_ids)
    
    for invoice_id in invoice_ids:
        agent.process_with_context(invoice_id, vendors)
```

### Monitoring and Observability

**Key Metrics to Track:**
```python
metrics = {
    "average_steps_per_invoice": 3.2,
    "success_rate": 0.95,
    "average_processing_time": 4.5,  # seconds
    "cost_per_invoice": 0.00125,     # USD
    "manual_review_rate": 0.08       # 8% require human review
}
```

**Error Patterns:**
```python
# Track what causes agents to fail
error_patterns = {
    "missing_vendor_data": 0.03,     # 3% of cases
    "invalid_payment_terms": 0.02,   # 2% of cases  
    "vat_validation_failure": 0.01,  # 1% of cases
    "llm_parsing_error": 0.005       # 0.5% of cases
}
```

### Scaling Considerations

**Horizontal Scaling:**
```python
# Multiple agents processing in parallel
from concurrent.futures import ThreadPoolExecutor

def process_invoices_parallel(invoice_ids, max_workers=5):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(agent.process, id) for id in invoice_ids]
        return [future.result() for future in futures]
```

In [ ]:
import time

# Simulate performance metrics
print("=" * 80)
print("PERFORMANCE METRICS")
print("=" * 80)

# Check server metrics if available
def get_server_metrics():
    try:
        response = requests.get(f"{OLLAMA_URL}/metrics")
        if response.status_code == 200:
            return response.json()
    except:
        pass
    return None

metrics = get_server_metrics()

if metrics:
    print("\n📊 Server Metrics:")
    print(f"GPU Memory Used: {metrics.get('gpu', {}).get('memory_used', 'N/A')} MB")
    print(f"GPU Memory Total: {metrics.get('gpu', {}).get('memory_total', 'N/A')} MB")
else:
    print("\n📊 Simulated Metrics:")

# Measure agent execution time
print("\n⏱️ Agent Performance:")

# Time a simple task
start_time = time.time()
test_agent = ReActAgent(mock_llm, tools, max_steps=5)
test_agent.run("Extract and validate invoice INV-2024-001")
execution_time = time.time() - start_time

print(f"\nTotal execution time: {execution_time:.2f} seconds")
print(f"Average time per step: {execution_time/4:.2f} seconds")

# Cost estimation
print("\n💰 Cost Analysis (Estimated):")
print(f"Model: {MODEL}")
print(f"Steps executed: 4")
print(f"Estimated tokens per step: ~500")
print(f"Total tokens: ~2000")
print(f"Estimated cost: ~$0.002 (at $0.001 per 1K tokens)")

print("\n🔍 Optimization Opportunities:")
print("1. Cache vendor history for repeated lookups")
print("2. Batch multiple invoice validations")
print("3. Use smaller model for simple extraction tasks")
print("4. Implement parallel tool execution")

## Key Learnings

### The ReAct Pattern Revolution

**What Makes It Powerful:**
- **Reasoning**: LLM thinks through the problem step by step
- **Acting**: Precise tool execution provides accurate data
- **Observing**: Results inform the next decision
- **Adapting**: Strategy changes based on discoveries

### Agent Architecture Principles

**Tool Design:**
```python
# Each tool should be:
1. Single-purpose and focused
2. Well-documented for LLM understanding  
3. Error-tolerant with graceful degradation
4. Return structured, parseable results
```

**Prompt Engineering:**
```python
# Effective ReAct prompts include:
1. Clear role definition
2. Available tools with descriptions
3. Response format specification
4. Context from previous steps
5. Specific output format requirements
```

**Error Handling:**
```python
# Production agents need:
1. Timeout and retry logic
2. Fallback strategies for service failures
3. Graceful degradation paths
4. Comprehensive error logging
5. Human escalation triggers
```

### Production Readiness Checklist

✅ **Performance**: Monitor token usage and execution time  
✅ **Reliability**: Handle service failures gracefully  
✅ **Observability**: Log all decisions and data used  
✅ **Security**: Validate inputs and sanitize outputs  
✅ **Scalability**: Design for parallel processing  
✅ **Cost Management**: Optimize prompt efficiency  

### When to Use ReAct Agents

**Perfect For:**
- Multi-step workflows with dependencies
- Processes requiring adaptation based on discovered data
- Tasks needing auditability and transparency
- Complex business rule application

**Not Ideal For:**
- Simple, linear data transformations
- Real-time applications requiring sub-second response
- Tasks with well-defined, unchanging workflows
- Situations where cost per operation is critical

### Next Steps: Advanced Patterns

In the next session, we'll explore:
- **LangGraph**: Building complex, stateful workflows
- **Parallel Execution**: Running multiple tools simultaneously
- **State Management**: Maintaining context across complex processes
- **Human-in-the-Loop**: Adding manual approval steps
- **Production Deployment**: Scaling agents in enterprise environments